In [0]:
%sql
CREATE OR REPLACE FUNCTION ankit_yadav.demo.get_accounts_by_account_executive(ae_name STRING)
RETURNS TABLE(
  account_id STRING,
  account_name STRING,
  account_executive_name STRING,
  account_executive_manager STRING,
  account_region STRING
)
COMMENT 'Returns selected account details managed by a specific Account Executive.'
RETURN
  SELECT
    account_id,
    account_name,
    account_executive_name,
    account_executive_manager,
    account_region
  FROM 
    ankit_yadav.demo.dim_accounts
  WHERE
    account_id IS NOT NULL
    AND account_executive_name = ae_name
  ORDER BY
    account_name;

In [0]:
%sql
SELECT * FROM ankit_yadav.demo.get_accounts_by_account_executive('David Wilson');

account_id,account_name,account_executive_name,account_executive_manager,account_region
ACCT001,Acme Corporation,David Wilson,Marcus Chen,West - Northern California


In [0]:
%sql
CREATE OR REPLACE FUNCTION ankit_yadav.demo.get_account_summaries(ae_name STRING)
RETURNS TABLE(
  Account STRING,
  AE STRING,
  AEM STRING,
  total_use_cases BIGINT,
  total_dollar_dbus DOUBLE,
  account_summary STRING
)
COMMENT 'Generates AI-powered account-level summaries for all accounts managed by a specific Account Executive. Analyzes use case patterns, pipeline health, opportunities, and risks across all use cases per account.'
RETURN 
  WITH account_use_cases AS (
    SELECT
      `account_executive` AS AE,
      `account_executive_manager` AS AEM,
      `account_name` AS Account,
      CONCAT_WS(
        '\n\n---\n\n',
        COLLECT_LIST(
          CONCAT(
            'Use Case Name: ', COALESCE(`use_case_name`, 'Unnamed'),
            '\nUse Case ID: ', COALESCE(`use_case_id`, 'N/A'),
            '\nUse Case Type: ', COALESCE(`use_case_type`, 'Not specified'),
            '\nDescription: ', COALESCE(`use_case_description`, 'No description'),
            '\nStage: ', COALESCE(`current_stage`, 'Unknown'),
            '\nMonthly Dollar DBUs: $', COALESCE(CAST(`value_dollars` AS STRING), '0'),
            '\nTarget Live Date: ', COALESCE(CAST(`target_live_date` AS STRING), 'Not set'),
            '\nDays in Current Stage: ', COALESCE(CAST(`current_stage_days` AS STRING), 'N/A'),
            '\nDays Since Last Modified: ', COALESCE(CAST(`days_since_last_modified` AS STRING), 'N/A'),
            '\nCreated Date: ', COALESCE(CAST(`created_date` AS STRING), 'Unknown')
          )
        )
      ) AS all_use_cases_text,
      COUNT(*) AS total_use_cases,
      SUM(COALESCE(`value_dollars`, 0)) AS total_dollar_dbus
    FROM
      `ankit_yadav`.`demo`.`dim_use_cases`
    WHERE
      `account_name` IS NOT NULL
      AND `account_executive` = ae_name
    GROUP BY
      `account_name`,
      `account_executive`,
      `account_executive_manager`
  )
  SELECT
    `Account`,
    `AE`,
    `AEM`,
    total_use_cases,
    total_dollar_dbus,
    ai_query(
      'databricks-gpt-oss-120b',
      CONCAT(
        'You are analyzing customer use cases for an Account Executive in the Manufacturing West region. ',
        'Below are ALL use cases for this customer account. ',
        'Please write a comprehensive account-level summary that:\n',
        '1. Identifies the main themes and patterns across all use cases\n',
        '2. Assesses the overall pipeline health and progression through stages\n',
        '3. Highlights high-value opportunities (based on Dollar DBUs) and quick wins\n',
        '4. Notes any risks, blockers, or use cases that need attention (e.g., stuck in stage for too long)\n',
        '5. Provides 3-5 strategic recommendations for the AE\n\n',
        'Account: ', `Account`, '\n',
        'Account Executive: ', `AE`, '\n',
        'Total Use Cases: ', CAST(total_use_cases AS STRING), '\n',
        'Total Monthly Dollar DBUs: $', CAST(total_dollar_dbus AS STRING), '\n\n',
        '--- USE CASES ---\n\n',
        all_use_cases_text
      )
    ) AS account_summary
  FROM
    account_use_cases
  ORDER BY
    total_dollar_dbus DESC,
    `Account`;


-- Test it directly in SQL
SELECT * FROM ankit_yadav.demo.get_account_summaries('David Wilson');

Account AE AEM total_use_cases total_dollar_dbus account_summary Acme Corporation David Wilson Marcus Chen 2 430000.0 **Acme Corporation – Account Summary (Manufacturing West)** 
**Account Executive:** David Wilson 
**Total Use Cases:** 2 
**Total Monthly DBU Commitment:** **$430 k** (≈ $5.16 M annualized) 

---

## 1. Core Themes & Patterns 

| Theme | Evidence from Use Cases | Business Impact |
|-------|--------------------------|-----------------|
| **Unified, real‑time analytics** | *Real‑time Data Lakehouse* – consolidates 12+ silos into a Delta Lake with sub‑minute latency. | Faster, cross‑functional decision making; reduces data duplication & governance overhead. |
| **Operationalizing AI/ML at scale** | *ML Pipeline Automation* – end‑to‑end model training, deployment, A/B testing, and registry. | Shortens time‑to‑value for predictive use cases (quality control, demand forecasting, predictive maintenance). |
| **Platform‑first, cloud‑native architecture** | Both use cases rely on Databricks Lakehouse, Delta Engine, MLflow, Jobs – a single, managed platform. | Lowers OPEX, simplifies ops, and creates a foundation for future data products. |
| **Manufacturing‑specific outcomes** | Implicit need for real‑time sensor/IoT data ingestion (lakehouse) and predictive models (ML). | Directly supports Acme’s goals of “smart factory” transformation and supply‑chain agility. |

*Pattern:* The two initiatives are tightly coupled – the lakehouse will feed the ML pipelines, and the ML pipelines will generate new analytics that the lakehouse will surface. This creates a **virtuous data‑product loop** that can be expanded to additional business units (e.g., finance, logistics) once the core platform is proven.

---

## 2. Pipeline Health & Stage Progression 

| Use Case | Stage (Numeric) | Days in Stage | Days Since Last Modified | Health Indicator |
|----------|----------------|---------------|--------------------------|------------------|
| **Real‑time Data Lakehouse** (UC001) | 3 – *Evaluating* | 14 | 2 | **On‑track** – early‑stage but moving quickly; recent activity suggests stakeholder engagement. |
| **ML Pipeline Automation** (UC002) | 4 – *Confirming* | 30 | 4 | **Stable but at risk of stall** – 30 days in “Confirming” is longer than typical (average 14‑21 days). Recent modification indicates some activity, but the stage is approaching a “stuck” threshold. |

**Overall pipeline health:** **Healthy to Moderate**. The total pipeline value is high, and the first use case is progressing smoothly. The second use case is the only potential drag point; if it does not move to “Closed‑Won” within the next 10‑14 days, the overall velocity could be perceived as slowing.

---

## 3. High‑Value Opportunities & Quick Wins 

| Opportunity | Monthly DBU Value | % of Total | Quick‑Win Potential | Why it matters |
|-------------|-------------------|-----------|---------------------|----------------|
| **Real‑time Data Lakehouse** | $250 k | 58 % | **High** – sub‑minute latency can be demonstrated with a pilot ingesting a single production line’s sensor data. | Immediate ROI for operations; strong showcase for the rest of the plant. |
| **ML Pipeline Automation** | $180 k | 42 % | **Medium** – a “single‑model” pilot (e.g., predictive maintenance on a critical asset) can be delivered in 4‑6 weeks once the lakehouse is live. | Demonstrates end‑to‑end value and accelerates the move from “Confirming” to “Closed‑Won”. |

**Quick‑win recommendation:** Prioritize a **“Lakehouse‑to‑ML pilot”** that uses the newly built lakehouse to feed a single ML model. This will give the customer a tangible success story, shorten the sales cycle for UC002, and reinforce the value of the larger $430 k/month commitment.

---

## 4. Risks, Blockers & Attention‑Needed Items 

| Risk / Blocker | Affected Use Case | Symptoms | Recommended Mitigation |
|----------------|-------------------|----------|------------------------|
| **Stage stagnation** | UC002 – *Confirming* | 30 

In [0]:
%sql
-- ============================================================================
-- FIXED FUNCTION: Using ARRAY_CONTAINS (More Reliable Alternative)
-- ============================================================================

CREATE OR REPLACE FUNCTION ankit_yadav.demo.get_live_date_follow_up_messages(ae_name STRING) 
RETURNS TABLE(
  Account STRING,
  AE STRING,
  AEM STRING,
  use_case_name STRING,
  use_case_id STRING,
  target_live_date DATE,
  days_until_live INT,
  current_stage STRING,
  monthly_dollar_dbus DOUBLE,
  follow_up_message STRING
) 
COMMENT 'Generates AI-powered follow-up messages for use cases targeting go-live in current or next month. 
Pass specific AE name, multiple names separated by comma, or NULL/"ALL" for all AEs. 
Creates concise Slack/Teams-ready messages for AE and SA based on next steps and use case details.'
RETURN 
  WITH ae_filter AS (
    SELECT ae_name as ae_input
  ),
  upcoming_use_cases AS (
    SELECT 
      `account_name`,
      `account_executive`,
      `account_executive_manager`,
      `use_case_name` AS use_case_name,
      `use_case_id` AS use_case_id,
      `target_live_date` AS target_live_date,
      DATEDIFF(`target_live_date`, CURRENT_DATE()) AS days_until_live,
      `current_stage` AS current_stage,
      `use_case_type` AS use_case_type,
      `use_case_description` AS use_case_description,
      `current_stage_days` AS days_in_current_stage,
      `value_dollars` AS monthly_dollar_dbus
    FROM 
      `ankit_yadav`.`demo`.`dim_use_cases`,
      ae_filter
    WHERE 
      -- Date filters
      `target_live_date` IS NOT NULL
      AND `target_live_date` >= DATE_TRUNC('MONTH', CURRENT_DATE())
      AND `target_live_date` < DATE_TRUNC('MONTH', ADD_MONTHS(CURRENT_DATE(), 2))
      AND `account_name` IS NOT NULL
      -- AE filter using ARRAY_CONTAINS instead of IN + LATERAL VIEW
      AND (
        ae_filter.ae_input IS NULL 
        OR TRIM(ae_filter.ae_input) = '' 
        OR UPPER(TRIM(ae_filter.ae_input)) = 'ALL'
        OR ARRAY_CONTAINS(
          TRANSFORM(SPLIT(ae_filter.ae_input, ','), x -> TRIM(x)),
          `account_executive`
        )
      )
  )
  SELECT 
    `account_name` AS Account,
    `account_executive` AS AE,
    `account_executive_manager` AS AEM,
    use_case_name,
    use_case_id,
    target_live_date,
    days_until_live,
    current_stage,
    monthly_dollar_dbus,
    ai_query(
      'databricks-gpt-oss-120b',
      CONCAT(
        'You are drafting a brief follow-up message for Slack/Teams to an Account Executive (AE) and Solutions Architect (SA). ',
        'The message should be conversational, action-oriented, and under 150 words.\n\n',
        'Context:\n',
        '- Account Executive: ', COALESCE(`account_executive`, 'Unknown AE'), '\n',
        '- Use Case: ', COALESCE(use_case_name, 'Unnamed Use Case'), '\n',
        '- Account: ', `account_name`, '\n',
        '- Target Live Date: ', CAST(target_live_date AS STRING), ' (', 
        CASE 
          WHEN days_until_live < 0 THEN CONCAT(ABS(days_until_live), ' days overdue')
          WHEN days_until_live = 0 THEN 'TODAY'
          WHEN days_until_live <= 7 THEN CONCAT(days_until_live, ' days away - URGENT')
          ELSE CONCAT(days_until_live, ' days away')
        END, ')\n',
        '- Current Stage: ', COALESCE(current_stage, 'Unknown'), '\n',
        '- Monthly Value: $', CAST(COALESCE(monthly_dollar_dbus, 0) AS STRING), '\n',
        '- Days in Stage: ', CAST(COALESCE(days_in_current_stage, 0) AS STRING), '\n',
        '- Use Case Type: ', COALESCE(use_case_type, 'Not specified'), '\n',
        '- Description: ', COALESCE(use_case_description, 'No description available'), '\n\n',
        'Instructions:\n',
        '1. Start with a friendly greeting addressing the AE by name (', COALESCE(`account_executive`, 'the AE'), ')\n',
        '2. Reference the use case name and target live date with urgency if close\n',
        '3. Ask specific questions to help move the use case forward\n',
        '4. If overdue or close to deadline, note the urgency tactfully\n',
        '5. Keep it under 150 words total\n',
        '6. End with a clear call-to-action\n',
        '7. Use a conversational, professional tone\n',
        '8. Do NOT include subject lines or email headers\n\n',
        'Write the message now:'
      )
    ) AS follow_up_message
  FROM 
    upcoming_use_cases
  ORDER BY 
    days_until_live ASC,
    monthly_dollar_dbus DESC;

# Testing

In [0]:
%sql
SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages('David Wilson') LIMIT 5;

Account,AE,AEM,use_case_name,use_case_id,target_live_date,days_until_live,current_stage,monthly_dollar_dbus,follow_up_message


In [0]:
%sql
-- ============================================================================
-- USAGE EXAMPLES
-- ============================================================================

-- Test with the test function first (no AI, faster)
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages_test('ALL') LIMIT 5;
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages_test(NULL) LIMIT 5;
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages_test('John Smith') LIMIT 5;
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages_test('John Smith, Jane Doe') LIMIT 5;

-- Once test function works, use the full version with AI
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages('ALL') LIMIT 5;

In [0]:
-- ============================================================================
-- USAGE EXAMPLES
-- ============================================================================

-- Example 1: Single AE
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages('John Smith');

-- Example 2: Multiple AEs (comma-separated)
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages('John Smith, Jane Doe, Bob Johnson');

-- Example 3: All AEs - Option 1 (NULL)
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages(NULL);

-- Example 4: All AEs - Option 2 ('ALL')
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages('ALL');

-- Example 5: All AEs - Option 3 (empty string)
-- SELECT * FROM ankit_yadav.demo.get_live_date_follow_up_messages('');

-- Example 6: Get summary by AE across all
-- SELECT 
--   AE,
--   AEM,
--   COUNT(*) as use_case_count,
--   SUM(monthly_dollar_dbus) as total_value,
--   COUNT(CASE WHEN days_until_live <= 7 THEN 1 END) as urgent_count,
--   COUNT(CASE WHEN days_until_live < 0 THEN 1 END) as overdue_count
-- FROM ankit_yadav.demo.get_live_date_follow_up_messages('ALL')
-- GROUP BY AE, AEM
-- ORDER BY urgent_count DESC, overdue_count DESC;

-- Example 7: Multiple AEs with filtering
-- SELECT * 
-- FROM ankit_yadav.demo.get_live_date_follow_up_messages('John Smith, Jane Doe')
-- WHERE days_until_live <= 14
-- ORDER BY AE, days_until_live;

-- Example 8: Get urgent use cases across all AEs for daily standup
-- SELECT 
--   AE,
--   Account,
--   use_case_name,
--   days_until_live,
--   CASE 
--     WHEN days_until_live < 0 THEN '🔴 OVERDUE'
--     WHEN days_until_live = 0 THEN '🔴 TODAY'
--     WHEN days_until_live <= 3 THEN '🟠 THIS WEEK'
--     ELSE '🟡 NEXT WEEK'
--   END as urgency_flag,
--   follow_up_message
-- FROM ankit_yadav.demo.get_live_date_follow_up_messages('ALL')
-- WHERE days_until_live <= 7
-- ORDER BY days_until_live, monthly_dollar_dbus DESC;